# Navigation

---

Congratulations for completing the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893)!  In this notebook, you will learn how to control an agent in a more challenging environment, where it can learn directly from raw pixels!  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.autograd import Variable
import torchvision.transforms.functional as F
from PIL import Image
from skimage.transform import resize
from collections import deque, namedtuple
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/VisualBanana.app"`
- **Windows** (x86): `"path/to/VisualBanana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/VisualBanana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/VisualBanana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/VisualBanana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/VisualBanana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/VisualBanana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `VisualBanana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="VisualBanana.app")
```

In [ ]:
env = UnityEnvironment(file_name="./VisualBanana_Linux/Banana.x86_64")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The environment state is an array of raw pixels with shape `(1, 84, 84, 3)`.  *Note that this code differs from the notebook for the project, where we are grabbing **`visual_observations`** (the raw pixels) instead of **`vector_observations`**.* A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.visual_observations[0]
print('States look like:')
plt.imshow(np.squeeze(state))
plt.show()
state_size = state.shape
print('States have shape:', state.shape)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.visual_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.visual_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from agent_pixels import DqnPixelAgent
from collections import deque
from model_pixels import *
import torch
import matplotlib.pyplot as plt
from skimage.transform import resize
from gym.spaces import Box
from gym.wrappers import FrameStack, GrayScaleObservation, TransformObservation
import cv2
# Gym is an OpenAI toolkit for RL
import gym
from pixel_utils import preprocess_state
import model_pixels
%matplotlib inline

In [ ]:
def dqn_pixel(agent, env, brain_name, n_episodes=5000,
        eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []  # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start  # initialize epsilon
    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.visual_observations[0]
        state = preprocess_state(state)
        score = 0
        while True:
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]
            next_state = env_info.visual_observations[0]  # get the next state
            next_state = preprocess_state(next_state)
            reward = env_info.rewards[0]  # get the reward
            done = env_info.local_done[0]  # see if episode has finished
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        scores_window.append(score)  # save most recent score
        scores.append(score)  # save most recent score
        eps = max(eps_end, eps_decay * eps)  # decrease epsilon
        print('\rEpisode {}\t Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode: {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>= 13.0:
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint_pixels.pth')
            break
    return scores

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]
# number of actions
action_size = brain.vector_action_space_size
# examine the state space
state = env_info.vector_observations[0]
state_size = len(state)
# create agent based on the config file in module config.agent_dqn_config.py
agent = DqnPixelAgent(state_size, action_size, seed=0)
# create agent-interface based on the config file in module config.agent_dqn_config.py
scores = dqn_pixel(agent, env, brain_name)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_size = 28

def process_state(x):
    x = np.squeeze(x)
    x_resize = resize(x, (image_size,image_size,3), anti_aliasing=True)
    x_modified = Variable(torch.from_numpy(x_resize).float().to(device).view(3,image_size,image_size))
    return x_modified

def process_stack(frames):
    frames_list = list(frames)
    torch_stack = torch.stack(frames_list).unsqueeze(dim=0)
    torch_stack = torch_stack.view(torch_stack.size(0),torch_stack.size(2), torch_stack.size(1), torch_stack.size(3), torch_stack.size(4) )
    return torch_stack


state_copy= []
def banana_dqn(episodes_num=5000, epsilon_start=1.0, epsilon_min=0.01, epsilon_decay=0.995, seed =0):
    scores = []
    score_window = deque(maxlen=100)
    epsilon = epsilon_start
    max_score = 7.0
    count = 5
    for episode in range(1, episodes_num+1):
        # reset the environment
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.visual_observations[0]            # get the current state
        score = 0
        stack_ready = False
        frame_stack= deque()
        stack_count = 0
        stack_size = 3
        is_state_processed = False
        while True:
#             state = resize(state, (state.shape[0],32, 32, 3), anti_aliasing=True)
            if not is_state_processed: 
                state = process_state(state)
            if len(frame_stack) < stack_size:
                
                for i in range(stack_size):
                    frame_stack.append(state)
                # for the first items just copy the times all at once
            else:
                frame_stack.popleft()
                frame_stack.append(state)
            processed_stack = process_stack(frame_stack)
            action = agent.act(processed_stack, epsilon)        # select an action
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.visual_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            score += reward                                # update the score
            next_state = process_state(next_state) # resize(next_state, (state.shape[0],32, 32, 3), anti_aliasing=True)
            
            next_fram_stack = frame_stack.copy()
            next_fram_stack.popleft()
            next_fram_stack.append(next_state)
            
            agent.step(state=processed_stack, action=action, reward=reward, next_state=process_stack(next_fram_stack), done=done, stack_size=stack_size)
            
            state = next_state 
            is_state_processed = True
            
            
            if done:
                # exit loop if episode finished
                break
            state_copy.append((state, action))
            count -= 1
            
        score_window.append(score)
        scores.append(score)
        epsilon = max(epsilon_min, epsilon*epsilon_decay)

        print('\rEpisode: {}\tAverage Score: {:.2f}'.format(episode, np.mean(score_window)), end="")
        if episode % 100 == 0:
            print('\rEpisode: {}\tAverage Score: {:.2f}'.format(episode, np.mean(score_window)))
        if np.mean(score_window)>= max_score:
            torch.save(agent.QN_local.state_dict(), 'checkpoint_pixels.pth')
            break
        
    return scores

In [ ]:
seed = 0
env_info = env.reset(train_mode=True)[brain_name]
action_size = brain.vector_action_space_size
state_size = len(env_info.vector_observations[0])
print('Action Size:{}, State Size:{}'.format(action_size, state_size))
agent = DqnPixelAgent(state_size, action_size, seed=0)
scores = banana_dqn(seed = seed)
#torch.save(agent.QN_local.state_dict(), 'checkpoint_pixels.pth')